<center><h1> Using FNN - Feed Forward Neural Network (LR/GBR)</center></h1>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow
import matplotlib.pyplot as plt
from sklearn import preprocessing,svm
from sklearn.linear_model import LinearRegression
import seaborn as sns
# example of making predictions for a regression problem
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from numpy import array
from random import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from keras import losses
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout
from sklearn.ensemble import GradientBoostingRegressor

### 1. FNN Architecture:
With this content, I using simple FNN with 2 hidden layer. 
- Layer 1: 1024 node
- Layer 2: 512 node
- Output: 1 node
<img src="./img/FNN.png" alt="drawing" style="width:400px;"/>
<center>FNN Architecture</center>

In [ ]:
def def_model():
    model = Sequential()
    model.add(Dense(1024, input_dim=10, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mse', # Mean squared error
                    optimizer='adam', # Optimization algorithm
                    metrics=['mse'])
    return model

### 2. Read data

In [ ]:
train = pd.read_csv("train.csv", sep=',')
test = pd.read_csv("test_id.csv",sep=',')

dropHeader = ['Before2Month','Before1Month','Before3Week','Before2Week','Before1Week']
dropAll = ['UPDATE_TIME','HOUR_ID','ZONE_CODE','SERVER_NAME','DayOfWeek','DayOfMonth','DayOfYear','And2MBefore','And1MBefore','And3WBefore','And2WBefore','And1WBefore']
dropAllLookup = ['UPDATE_TIME','HOUR_ID','ZONE_CODE','SERVER_NAME','DayOfWeek','DayOfMonth','DayOfYear','And2MBefore','And1MBefore','And3WBefore','And2WBefore','And1WBefore','Traffic1WBefore','Traffic2WBefore','Traffic3WBefore','Traffic1MBefore','Traffic2MBefore']
feature_output = ['MAX_USER','BANDWIDTH_TOTAL']
feature_input = ['HOUR_ID','ZONE_CODE','DayOfWeek','DayOfMonth','DayOfYear','Traffic1WBefore','Traffic2WBefore','Traffic3WBefore','Traffic1MBefore','Traffic2MBefore']
all_server_name = test.SERVER_NAME.unique()

In [ ]:
all_server_name = all_server_name[0:3] #Only for test with small data
all_server_name

### 3. Preprocessing data and make feature:
Input of FNN have 10 feature as below:
- HOUR_ID:
- ZONE_CODE: with Label Encoder
- DayOfWeek: Day of week
- DayOfMonth: Day of month
- DayOfYear: Day of year
- Before2Month: Data collected from the last 2 months
- Before1Month: Data collected from the last previous months
- Before3Week: Data collected from the last 3 weeks
- Before2Week: Data collected from the last 2 weeks
- Before1Week: Data collected from the last week

In [ ]:
for y1 in feature_output:
    print(y1)
    train_data = train[['UPDATE_TIME','HOUR_ID','ZONE_CODE','SERVER_NAME',y1]]
    test_data = test[['UPDATE_TIME','HOUR_ID','ZONE_CODE','SERVER_NAME']]
    test_data[y1] = 0
    
    #concat train & test to process one time
    all_data = pd.concat([train_data,test_data])
    
    #Day of Week/Month/Year
    all_data['UPDATE_TIME'] = pd.to_datetime(all_data['UPDATE_TIME'])
    all_data['DayOfWeek'] = all_data['UPDATE_TIME'].dt.dayofweek
    all_data['DayOfMonth'] = all_data['UPDATE_TIME'].dt.day
    all_data['DayOfYear'] = all_data['UPDATE_TIME'].dt.dayofyear
    
    #Shift time
    all_data['Before2Month']=all_data.UPDATE_TIME.apply(lambda x: x + relativedelta(months=-2))
    all_data['Before1Month']=all_data.UPDATE_TIME.apply(lambda x: x + relativedelta(months=-1))
    all_data['Before3Week']=all_data.UPDATE_TIME.apply(lambda x: x + timedelta(days=-7*3))
    all_data['Before2Week']=all_data.UPDATE_TIME.apply(lambda x: x + timedelta(days=-7*2))
    all_data['Before1Week']=all_data.UPDATE_TIME.apply(lambda x: x + timedelta(days=-7*1))
    
    #Make data point
    all_data['AndFirst'] = all_data['UPDATE_TIME'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data['And2MBefore'] = all_data['Before2Month'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data['And1MBefore'] = all_data['Before1Month'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data['And3WBefore'] = all_data['Before3Week'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data['And2WBefore'] = all_data['Before2Week'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data['And1WBefore'] = all_data['Before1Week'].map(str) + all_data['HOUR_ID'].map(str) +  all_data['SERVER_NAME'].map(str)
    all_data = all_data.drop(dropHeader,axis=1)
    dataX = all_data.drop(dropAll, axis=1)
    
    #Lookup data
    for i in range(1,4):
        strTraffic = 'Traffic' + str(i) + 'WBefore'
        strBeffore = 'And'+ str(i) + 'WBefore'
        trainReplace = dataX.rename(columns={y1: strTraffic, 'AndFirst': strBeffore})
        all_data = all_data.merge(trainReplace, on=strBeffore, how='left')
    for i in range(1,3):
        strTraffic = 'Traffic' + str(i) + 'MBefore'
        strBeffore = 'And'+ str(i) + 'MBefore'
        trainReplace = dataX.rename(columns={y1: strTraffic, 'AndFirst': strBeffore})
        all_data = all_data.merge(trainReplace, on=strBeffore, how='left')
    
    all_data = all_data.dropna(axis=0)
    #Label Encoder
    le = preprocessing.LabelEncoder()
    le.fit(all_data['ZONE_CODE'])
    all_data['ZONE_CODE'] = le.transform(all_data['ZONE_CODE']) 
    all_data.to_csv("FNN_DataOK_" + y1 + ".csv", index=False)
    print('Export: ', "FNN_DataOK_" + y1 + ".csv", all_data.shape)
    del all_data


### 4. Predict
- Because we must predict 1 month for every server. This is long time.
- We will predict 1 week in future. When we have result, we will concat with input data to make input feature for predict next week.
<img src="./img/time_array.PNG" alt="drawing" style="width:100%;"/>

In [ ]:
test_data_x = test.copy()
test_data_x.UPDATE_TIME = pd.to_datetime(test_data_x.UPDATE_TIME)
test_data_x["AndFirst"] = test_data_x.UPDATE_TIME.map(str) + test_data_x.HOUR_ID.map(str) + test_data_x.SERVER_NAME.map(str)

for y1 in feature_output:
    test_data = test[['UPDATE_TIME','HOUR_ID','ZONE_CODE','SERVER_NAME']]
    test_data[y1] = 0
    all_data = pd.read_csv("FNN_DataOK_" + y1 + ".csv")
    result = all_data.head(1)
    print('--------------- Run: ', y1,all_data.shape, '---------------------')
    for item in all_server_name:
        data_one = all_data[all_data.SERVER_NAME == item]
        print(item,data_one.shape)
        data_one['UPDATE_TIME'] = pd.to_datetime(data_one['UPDATE_TIME'])
        train_model_one = data_one[data_one.UPDATE_TIME < '2019-03-10']
        train_model_one_x = train_model_one[feature_input]
        train_model_one_y = train_model_one[[y1]]
        train_model_one_y = np.asarray(train_model_one_y).reshape(-1,1)
        
        #Normalization data
        scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
        scalarX.fit(train_model_one_x)
        scalarY.fit(train_model_one_y)
        train_model_one_xs = scalarX.transform(train_model_one_x)
        train_model_one_ys = scalarY.transform(train_model_one_y)
        
        #Split train & validation
        train_one_xs_new, validate_one_xs, train_one_ys_new, validate_one_ys = train_test_split(train_model_one_xs, train_model_one_ys, test_size=0.1, random_state=0)
        accuracy = 0
        
#         #LINEAR REGRESSION MODEL
#         model=LinearRegression()
#         model.fit(train_one_xs_new,train_one_ys_new)
#         accuracy = model.score(validate_one_xs,validate_one_ys)

#         #GradientBoostingRegressor
#         model = GradientBoostingRegressor(n_estimators=100, max_depth=3, loss='ls')
#         model.fit(train_model_one_xs, train_model_one_ys)
        
        #FNN
        model = def_model()
        model.fit(train_one_xs_new, 
                  train_one_ys_new, 
                  batch_size=256, 
                  epochs=5, 
                  validation_data=(validate_one_xs, validate_one_ys))
        
        date_time_test = '2019-03-10'
        start_date = datetime.strptime(date_time_test, '%Y-%m-%d')
        max_date = datetime.strptime('2019-04-10', '%Y-%m-%d')
        
        end_date = start_date
        while(end_date < max_date):
            #Predict one week
            end_date = start_date + timedelta(days=+7)
            if (end_date > max_date): 
                end_date = max_date
            print(item,accuracy,start_date,"-",end_date)
            dataX_one_data = data_one.drop(dropAllLookup, axis=1)
            dataX_one_data.head()
            dataX_one_data_rmv = dataX_one_data.drop_duplicates()
            test_one = data_one[(data_one.UPDATE_TIME < end_date) & (data_one.UPDATE_TIME >= start_date)]
            if (test_one.shape[0] > 0):
                test_one = test_one.drop(['Traffic1WBefore','Traffic2WBefore','Traffic3WBefore','Traffic1MBefore','Traffic2MBefore'], axis=1)
                for i in range(1,4):
                    strTraffic = 'Traffic' + str(i) + 'WBefore'
                    strBeffore = 'And'+ str(i) + 'WBefore'
                    trainReplace = dataX_one_data_rmv.rename(columns={y1: strTraffic, 'AndFirst': strBeffore})
                    test_one = test_one.merge(trainReplace, on=strBeffore, how='left')
                for i in range(1,3):
                    strTraffic = 'Traffic' + str(i) + 'MBefore'
                    strBeffore = 'And'+ str(i) + 'MBefore'
                    trainReplace = dataX_one_data_rmv.rename(columns={y1: strTraffic, 'AndFirst': strBeffore})
                    test_one = test_one.merge(trainReplace, on=strBeffore, how='left')
                test_one_x = test_one[feature_input]

                test_one_x[test_one_x == ""] = np.NaN
                test_one_x = test_one_x.fillna(method='ffill')
                test_one_x = test_one_x.fillna(0)
                test_one_x = scalarX.transform(test_one_x)

                test_one_y = model.predict(test_one_x)

                test_one_y_inverse = scalarY.inverse_transform(test_one_y.reshape(-1,1))
                data_one.loc[(data_one.UPDATE_TIME < end_date) & (data_one.UPDATE_TIME >= start_date), y1] = test_one_y_inverse
            else:
                print("No data - ", item)
            start_date = end_date
        
        
        # concat data
        result = pd.concat([result, data_one[data_one.UPDATE_TIME >= date_time_test]])
        
        # clear data    
        del data_one 
        del train_model_one 
        del train_model_one_x 
        del train_model_one_y
        del train_model_one_xs
        del train_model_one_ys
        del model
        
    result.to_csv('RESULT_' + y1 + '.csv')
    lookup_data = result[['AndFirst',y1]]
    test_data_x = test_data_x.merge(lookup_data,on='AndFirst',how='left')
    del result
    del all_data


### 5. Output & Trick
Metric of this contest is MAPE:
<img src="./img/MAPE.PNG" alt="drawing" style="width:300px;"/>

So, when truth label At is so small, we maybe get MAPE that is so high. To avoid this case, when value of predict so small we set equal 0.

In [ ]:
test_data_x[test_data_x == ""] = np.NaN
test_data_x = test_data_x.fillna(method="ffill")
test_data_x.to_csv("Backup_Submit.csv", index=False)
test_data_x_copy = test_data_x[['id', 'BANDWIDTH_TOTAL', 'MAX_USER']].copy()
test_data_x_copy.MAX_USER[test_data_x_copy.MAX_USER < 1] = 0
test_data_x_copy.BANDWIDTH_TOTAL[test_data_x_copy.BANDWIDTH_TOTAL < 260] = 0
test_data_x_copy['label'] = round(test_data_x_copy.BANDWIDTH_TOTAL,2).map(str) + " " + round(test_data_x_copy.MAX_USER,2).map(str)
test_data_x_copy[['id', 'label']].to_csv("FNN_Submit.csv", index=False)